```
load the classifier and evaluate it on the test set
```

In [17]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import re
import cv2
from PIL import Image

import datetime

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 600
batchsize = 32
testbatchsize = 16


In [19]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [20]:
class My_dataset(Dataset):
    def __init__(self, train):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        dir = os.listdir("./grids_single_trainset")
        dirtest = os.listdir("./grids_eval")
        data = []
        a = []
        if train:
            # for metric in dir:
            #     data.append(
            #         [np.loadtxt("./grids_trainset_cifar/"+metric), metric])
            sample = []
            for metric in dir:
                sample.append([np.loadtxt("./grids_single_trainset/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []
            
        else:
            # for metric in dirtest:
            #     data.append(
            #         [np.loadtxt("./grids_testset_cifar/"+metric), metric])
            sample = []
            for metric in dirtest:
                sample.append([np.loadtxt("./grids_eval/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []

        self.x = [item[0] for item in data]
        self.y = [int(item[1][0:len("h_vr_metric-single-leNet_morefc-continue=mnist_moreFC_global-2_attack_8")] == "h_vr_metric-single-leNet_morefc-continue=mnist_moreFC_global-2_attack_8") for item in data]
        # self.y = [int(re.findall("\d+\.?\d*", item[1])[0]) for item in data]
        self.src,  self.trg = [], []
        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [21]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
     
data_test = My_dataset(train=False)
data_loader_test = DataLoader(data_test, batch_size=testbatchsize, shuffle=False)


print("test")
for i_batch, batch_data in enumerate(data_loader_test):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg


test
0
torch.Size([12, 4, 128, 128])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])


In [22]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=64),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16 * 32 * 32, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = MyNet().to(device)
model.load_state_dict(torch.load('./classifier/single=07-18--21-27-13.pth')["state_dict"])

<All keys matched successfully>

In [23]:
# define cost/loss & optimizer
# Softmax is internally computed.
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [24]:
# Test the model using test sets
# model.load_state_dict(torch.load(".\merge_save\merge_attack0normal10_05-07--21-10-32.pth")["state_dict"])
# model.load_state_dict(torch.load(r".\myfed_normal_save\model18-24-02.pth")["state_dict"])
model.eval()
with torch.no_grad():
    for X_test, Y_test in data_loader_test:
        X_test = X_test.to(device).float()
        Y_test = Y_test.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        print(prediction,correct_prediction, Y_test)
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

    # Get one and predict
    # r = random.randint(0, len(mnist_test) - 1)
    # X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    # Y_single_data = mnist_test.targets[r:r + 1].to(device)

    # print('Label: ', Y_single_data.item())
    # single_prediction = model(X_single_data)
    # print('Prediction: ', torch.argmax(single_prediction, 1).item())


tensor([[ 3.9552, -3.9408],
        [ 0.1528, -0.2748],
        [ 4.3995, -4.5735],
        [-2.9087,  2.4308],
        [ 1.0717, -1.3018],
        [ 0.0122, -0.0246],
        [ 0.1151, -0.2514],
        [-0.4324,  0.1780],
        [ 0.3071, -0.4691],
        [ 0.0660, -0.3785],
        [ 1.0297, -1.2225],
        [-0.7202,  0.3263]], device='cuda:0') tensor([ True,  True,  True, False,  True,  True,  True,  True, False, False,
        False,  True], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], device='cuda:0')
Accuracy: 0.6666666865348816
